<a href="https://colab.research.google.com/github/kupieckozienice/pyPDF2/blob/main/pypdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[LIBRARY] zainstalowanie biblioteki PyPDF2

In [2]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.2 MB/s eta 0:00:00


[LIBRARY] install pdfminer.six

In [3]:
pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 12.5 MB/s eta 0:00:00


[LIBRARY] importowanie niezbędnych *bibliotek*

In [29]:
import os
import pathlib as pth
from typing import Union, Literal, List
from PyPDF2 import PdfReader, PdfWriter

[DATA] konwersja daty

In [5]:
def convertPdfDatetime(pd):
    from datetime import datetime
    dtformat = "%Y%m%d%H%M%S"
    clean = pd.replace("D:","").split('+')[0]
    return datetime.strptime(clean,dtformat)

[PDF] add blank page

In [6]:
# add blank page
def add_blank_page(writer, path_destination):
  writer.add_blank_page(width=200, height=200)

  data = b"any bytes - typically read from a file"
  writer.add_attachment("smile.png", data)

  with open("output.pdf", "wb") as output_stream:
      writer.write(output_stream)

[PDF] sprawdzenie czy jest zaszyfrowany hasłem,
jeśli tak to należy do odszyfrować pobranym hasłem



In [31]:
def check_if_encrypted(path_to_file):
  if os.path.isfile(path_to_file):
    reader = PdfReader(path_to_file)

    if reader.is_encrypted:
      writer = PdfWriter()
      password = input("Enter your password: ")
      reader.decrypt(password)

      # Add all pages to the writer
      for page in reader.pages:
          writer.add_page(page)

      # Save the new PDF to a file
      with open(path_to_file, "wb") as f:
          writer.write(f)

      print("plik rozszyfrowany")
    else:
      print("plik NIE był zaszyfrowany")

[PDF] czytanie adnotacji

In [8]:
def annots_reading(reader):
  print("funkcja annots_reading załadowana")

  attachments = {}

  for page in reader.pages:
    if "/Annots" in page:
        for annot in page["/Annots"]:
            obj = annot.get_object()
            annotation = {"subtype": obj["/Subtype"], "location": obj["/Rect"]}
            print(annotation)

            # text
            subtype = annot.get_object()["/Subtype"]
            if subtype == "/Text":
                print(annot.get_object()["/Contents"])

            # Highlights
            subtype = annot.get_object()["/Subtype"]
            if subtype == "/Highlight":
                coords = annot.get_object()["/QuadPoints"]
                x1, y1, x2, y2, x3, y3, x4, y4 = coords

            # # attachments
            # subtype = annot.get_object()["/Subtype"]
            # if subtype == "/FileAttachment":
            #     fileobj = annotobj["/FS"]
            #     attachments[fileobj["/F"]] = fileobj["/EF"]["/F"].get_data()
    else:
      # print("brak andotacji w pliku")
      pass

[PYTHON] iterate over files in directory

In [9]:
# iterate over files in directory

def iterate_over_files_in_directory(path_to_directory):

  numerate = 1

  for filename in os.listdir(path_to_directory):
    f = os.path.join(path_to_directory, filename)
    if os.path.isfile(f):
      print(f"{numerate}. {f}")
      numerate = numerate + 1

[PDF] add the metadata

In [10]:
# Add the metadata
def add_metadata(reader):
  writer = PdfWriter()

  # Add all pages to the writer
  for page in reader.pages:
      writer.add_page(page)

  writer.add_metadata(
      {
          "/Author": "Martino",
          "/Producer": "Libre Writer",
          "/Subject": "kikk",
          "/Title": "jakis_tytul",
          # "/ModDate": modification_date  # Adding modification date
      }
  )

  with open(os.path.join(path_to_working_directory, file_name), "wb") as f:
    writer.write(f)


[PDF] read metadata

In [11]:
def read_metadata(reader):
  meta = reader.metadata
  print(len(reader.pages))

  # All of the following could be None!
  print(f"author: {meta.author}")
  print(f"creator: {meta.creator}")
  print(f"producer: {meta.producer}")
  print(f"subject: {meta.subject}")
  print(f"title: {meta.title}")

[PDF] reading form fields

In [12]:
# reading form fields

def reading_form_fields(reader):
  # fields = reader.get_form_text_fields()
  # fields == {"key": "value", "key2": "value2"}
  # if reader.isEditable:
  fields = reader.get_fields()

  if bool(fields):
    # print(fields)
    for field_name, field_value in fields.items():
      print(f"nazwa pola: {field_name}, wartość: {field_value}")
  else:
    print("brak fields-ów")

  # for field_name in fields:
    # print(field_name)

[PDF] extract text

In [13]:
# extract text
def extract_text(reader):
  text = ""
  try:
      for page in reader.pages:
          text += page.extract_text()
  except Exception as exc:
      text = fallback_text_extraction("example.pdf")

  return text

[PYTHON] sprawdzenie czy istnieje katalog roboczy

In [51]:
# sprawdzenie czy istnieje katalog roboczy, w którym chcemy pracować
def check_if_directory_and_path_exists(directory_name):
  # Ścieżka do katalogu bieżącego
  cwd = os.getcwd()
  print(f"katalog \"bieżący\": {cwd}")

  working_directory_for_creating = os.path.join(cwd, directory_name)
  print(f"katalog roboczy do utworzenia (jeśli nie istnieje): {working_directory_for_creating}")

  try:
    if not os.path.exists(working_directory_for_creating):
      print(f"katalog: {working_directory_for_creating} nie istnieje i zostanie utworzony")
      os.makedirs(working_directory_for_creating)
    else:
      print(f"katalog: {working_directory_for_creating} już istnieje!")
  except FileExistsError:
    print(f"katalog o nazwie {working_directory_for_creating} już istnieje!")
  except Exception as e:
    print(f"wystąpił błąd podczas tworzenie katalogu: {working_directory_for_creating}")

[PYTHON] upload plików z dysku lokalnego

In [140]:
def import_files(destination_directory):
  from google.colab import files

  uploaded = files.upload()
  # print(uploaded.keys())

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

  for filename, file_content in uploaded.items():
    if os.path.exists(destination_directory) and os.path.isdir(destination_directory):
      with open(os.path.join(destination_directory, filename), "wb") as f:
        f.write(file_content)

      # remove files from /content
      file_to_remove_after_copying_to_working_directory = os.path.join(os.getcwd(), filename)
      if os.path.isfile(file_to_remove_after_copying_to_working_directory):
        os.remove(file_to_remove_after_copying_to_working_directory)
        if not os.path.exists(file_to_remove_after_copying_to_working_directory):
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")
        else:
          print(f"plik: {os.path.basename(file_to_remove_after_copying_to_working_directory)} "
          f"NIE usunięty z katalogu {os.path.dirname(file_to_remove_after_copying_to_working_directory)}")



In [239]:
# usuwanie stron w pdf
def page_deleting(source_file, pages_nums_to_delete):
  print(source_file)

  file_name_without_ext, ext = os.path.splitext(source_file)
  new_file_name = ''.join([os.path.basename(file_name_without_ext), "_with_deleted_pages", ext])
  destination_file = os.path.join(os.path.dirname(source_file), new_file_name)
  print(file_name_without_ext)
  print(ext)
  print(new_file_name)

  with open(source_file, "rb") as file:
    reader = PdfReader(file)
    writer = PdfWriter()

    for page_num in range(len(reader.pages)):
      if page_num + 1 not in pages_nums_to_delete:
        page_to_stay = reader.pages[page_num]
        writer.add_page(page_to_stay)

  with open(destination_file, "wb") as new_file:
    writer.write(new_file)

  with open(destination_file, "rb") as new_file:
    print(f"liczba stron nowego pliku: {len(PdfReader(new_file).pages)}")


main app

In [242]:
from datetime import datetime  # Import the datetime module
from pdfminer.high_level import extract_text as fallback_text_extraction

# Ścieżka do katalogu, w którym będą uploudowene pliki
path_to_working_directory = os.path.join(os.getcwd(), "pypdf2")
print(f"Ścieżka do katalogu, w którym będą uploudowene pliki: {path_to_working_directory}")

# sprawdzenie czy istnieje katalog roboczy, w którym chcemy pracować
check_if_directory_and_path_exists(path_to_working_directory)

# usuwanie stron
page_to_delete = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
page_deleting("pypdf2/oferta_handlowa_wzor.pdf", page_to_delete)

# upload plików roboczych
# import_files(path_to_working_directory)

# reader = PdfReader("/content/pypdf2/oferta_handlowa_wzor.pdf")
# writer = PdfWriter()
# check_if_encrypted("/content/pypdf2/oferta_handlowa_wzor.pdf")

# annots reading
# annots_reading(reader)

# czytanie metadanych
# meta = reader.metadata
# meta

# iterate over files in directory
# iterate_over_files_in_directory(path_to_working_directory)

# Reading form fields
# reading_form_fields(reader)

# extract text
# text = extract_text(reader)
# print(text)

# add metadata
# add_metadata(reader)

Ścieżka do katalogu, w którym będą uploudowene pliki: /content/pypdf2
katalog "bieżący": /content
katalog roboczy do utworzenia (jeśli nie istnieje): /content/pypdf2
katalog: /content/pypdf2 już istnieje!
pypdf2/oferta_handlowa_wzor.pdf
pypdf2/oferta_handlowa_wzor
.pdf
oferta_handlowa_wzor_with_deleted_pages.pdf
liczba stron nowego pliku: 1
